In [1]:
# Library Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import optuna
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_squared_error
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import KFold  # Assuming no stratification needed for regression
import gc
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping
import re
import polars as pl
from catboost import CatBoostRegressor, Pool
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error

pd.set_option('future.no_silent_downcasting', True)

In [2]:
train_data=pl.read_csv("/kaggle/input/brist1d/train.csv")
train_data=train_data.to_pandas()
test_data=pl.read_csv("/kaggle/input/brist1d/test.csv")
test_data=test_data.to_pandas()
print(len(train_data),len(test_data))
train_data.head()

177024 3644


,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,None,NaN,9.6,None,NaN,9.7,None,...,None,None,None,None,None,None,None,None,None,13.4
1,p01_1,p01,06:25:00,None,NaN,9.7,None,NaN,9.2,None,...,None,None,None,None,None,None,None,None,None,12.8
2,p01_2,p01,06:40:00,None,NaN,9.2,None,NaN,8.7,None,...,None,None,None,None,None,None,None,None,None,15.5
3,p01_3,p01,06:55:00,None,NaN,8.7,None,NaN,8.4,None,...,None,None,None,None,None,None,None,None,None,14.8
4,p01_4,p01,07:10:00,None,NaN,8.4,None,NaN,8.1,None,...,None,None,None,None,None,None,None,None,None,12.7


In [3]:
test_id = test_data["id"]

In [4]:
feats=['bg','insulin','hr','steps','cals','activity']
ts=['-5:55', '-5:50', '-5:45', '-5:40', '-5:35', '-5:30', '-5:25', '-5:20', 
    '-5:15', '-5:10', '-5:05', '-5:00', '-4:55', '-4:50', '-4:45', '-4:40', 
    '-4:35', '-4:30', '-4:25', '-4:20', '-4:15', '-4:10', '-4:05', '-4:00', 
    '-3:55', '-3:50', '-3:45', '-3:40', '-3:35', '-3:30', '-3:25', '-3:20', 
    '-3:15', '-3:10', '-3:05', '-3:00', '-2:55', '-2:50', '-2:45', '-2:40', 
    '-2:35', '-2:30', '-2:25', '-2:20', '-2:15', '-2:10', '-2:05', '-2:00', 
    '-1:55', '-1:50', '-1:45', '-1:40', '-1:35', '-1:30', '-1:25', '-1:20', 
    '-1:15', '-1:10', '-1:05', '-1:00', '-0:55', '-0:50', '-0:45', '-0:40', 
    '-0:35', '-0:30', '-0:25', '-0:20', '-0:15', '-0:10', '-0:05', '-0:00']

def get_more_data(df,timestep,mode='train'):#mode is train or test

    #12step is 1 hour.
    start = 0
    fin = len(ts)
    end = timestep
    target = end + 11
    start_2 = fin - end
    df['time'] = pd.to_timedelta(df["time"])
    
    df["time"] = [td.total_seconds() / 3600 for td in df["time"]]
        
    
    if mode == "train":
        new_df=df[['p_num']+["time"] + [f+ts[i]  for i in range(start_2, fin) for f in feats]+['bg+1:00']]
        new_df.columns=['p_num']+ ["time"] +[f+str(i)  for i in range(start, end)for f in feats]+['bg+1:00']

        df_2 = df[['p_num']+ ["time"] + [f+ts[i]  for i in range(start, end) for f in feats]+['bg'+ts[target]]]
        df_2.columns=['p_num']+ ["time"] + [f+str(i)  for i in range(start, end)for f in feats]+['bg+1:00']
        df_2["time"] = (df_2["time"] - 1) % 24
    if mode == "test":
        new_df=df[['p_num']+["time"] + [f+ts[i]  for i in range(start_2, fin) for f in feats]]
        new_df.columns=['p_num']+["time"] + [f+str(i)  for i in range(start, end)for f in feats]
        
        df_2 = df[['p_num']+["time"] + [f+ts[i]  for i in range(start, end) for f in feats]+['bg'+ts[target]]]
        df_2.columns=['p_num']+["time"] + [f+str(i)  for i in range(start, end)for f in feats]+['bg+1:00']
        df_2["time"] = (df_2["time"] - 1) % 24

        
              
    new_df = [new_df, df_2]
    new_df = pd.concat(new_df)
    
    bins = [0, 4, 12, 18, 24]  # Bin edges
    labels = ["Night", "Morning", "Afternoon", "Night"]  # Corresponding labels
    
    new_df['time'] = pd.cut(new_df['time'], bins=bins, labels=labels, right=True, include_lowest=True, ordered=False)

    return new_df

In [5]:
def FE(df, mode='train', timestep=36):
    print("< drop useless columns >")
    df.drop([c for c in df.columns if 'carbs' in c] + ['id'], axis=1, inplace=True)

    print("< fillna >")
    bgs = [c for c in list(df.columns) if (c != 'bg+1:00') and ('bg' in c)]
    df[bgs] = df[bgs].ffill(axis=1)
    df[bgs] = df[bgs].bfill(axis=1)

    print("< log1p transform >")
    step_cols = [c for c in list(df.columns) if 'step' in c]
    df[step_cols] = df[step_cols].fillna(50).astype(np.float32)
    df[step_cols] = np.log1p(df[step_cols])
    
    print("< fix bug >")
    float_cols = [c for c in df.columns for f in feats[:-1] if f in c]
    df[float_cols] = df[float_cols].astype(np.float32)

    print("< get more data >")
    df = get_more_data(df, timestep=timestep, mode=mode)

    df['p_num_int'] = df['p_num'].apply(lambda x: int(x[1:])).astype(np.int8)

    print("< fill missing feature values >")
    float_cols = [col + str(ts) for ts in range(timestep) for col in ['bg', 'insulin', 'hr', 'steps', 'cals']]
    df[float_cols] = df[float_cols].fillna(-1)

    print("< activity feature >")
    activity_cols = [c for c in list(df.columns) if 'activity' in c]
    activity_feats = df[activity_cols].replace('None', '').fillna('')
    df['activity_present'] = activity_feats.apply(lambda row: any(row), axis=1).astype(int)
    df.drop(activity_cols, axis=1, inplace=True)

    print("< Z-score feature >")
    cols = ['bg', 'insulin', 'steps', 'hr', 'cals']

    # Create columns for timestep suffix only once
    suffix = f"{timestep - 1}"
    cols_with_suffix = [f"{col}{suffix}" for col in cols]

    # Group by `p_num` and calculate mean and std for each feature at the given timestep
    grouped_stats = df.groupby('p_num')[cols_with_suffix].agg(['mean', 'std']).reset_index()
    grouped_stats.columns = ['p_num'] + [f"{stat}_pnum_{col}" for col in cols_with_suffix for stat in ['mean', 'std']]

    # Join the mean and std columns back to the original DataFrame on `p_num`
    df = df.merge(grouped_stats, on='p_num', how='left')

    # Calculate z-scores for each column using the joined mean and std
    z_score_features = {}
    for col in cols:
        mean_col = f'mean_pnum_{col}{suffix}'
        std_col = f'std_pnum_{col}{suffix}'
        z_score_features[f'norm_pnum_{col}{suffix}'] = (df[f'{col}{suffix}'] - df[mean_col]) / df[std_col]

    # Convert z-score calculations to DataFrame and join all at once to avoid fragmentation
    z_score_df = pd.DataFrame(z_score_features)
    df = pd.concat([df, z_score_df], axis=1)

    print("< cross feature >")
    cross = ['bg', 'insulin', 'steps', 'hr', 'cals']
    for i in range(len(cross)):
        for j in range(i + 1, len(cross)):
            for k in [1]:
                df[f"{cross[i]}{timestep - k}_{cross[j]}{timestep - k}"] = df[f"{cross[i]}{timestep - k}"] * df[f"{cross[j]}{timestep - k}"]

    return df



In [6]:
train = FE(train_data, mode='train')
test = FE(test_data, mode='test')
total = pd.concat((train, test))
test = total[total['bg+1:00'].isna()]
train = total[~total['bg+1:00'].isna()]
test.drop(["bg+1:00"], axis=1, inplace=True)

print(len(train), len(test), len(total))
train.head()

< drop useless columns >
< fillna >
< log1p transform >
< fix bug >
< get more data >


/tmp/ipykernel_23/2670890814.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["time"] = (df_2["time"] - 1) % 24
/tmp/ipykernel_23/581369802.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['p_num_int'] = df['p_num'].apply(lambda x: int(x[1:])).astype(np.int8)


< fill missing feature values >
< activity feature >


/tmp/ipykernel_23/581369802.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['activity_present'] = activity_feats.apply(lambda row: any(row), axis=1).astype(int)


< Z-score feature >
< cross feature >
< drop useless columns >
< fillna >
< log1p transform >
< fix bug >
< get more data >
< fill missing feature values >
< activity feature >


/tmp/ipykernel_23/2670890814.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["time"] = (df_2["time"] - 1) % 24
/tmp/ipykernel_23/581369802.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['p_num_int'] = df['p_num'].apply(lambda x: int(x[1:])).astype(np.int8)
/tmp/ipykernel_23/581369802.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inst

< Z-score feature >
< cross feature >
357692 3644 361336


/tmp/ipykernel_23/2553637795.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(["bg+1:00"], axis=1, inplace=True)


,p_num,time,bg0,insulin0,hr0,steps0,cals0,bg1,insulin1,hr1,...,bg35_insulin35,bg35_steps35,bg35_hr35,bg35_cals35,insulin35_steps35,insulin35_hr35,insulin35_cals35,steps35_hr35,steps35_cals35,hr35_cals35
0,p01,Morning,13.900000,0.0083,-1.0,3.931826,-1.0,13.900000,0.0083,-1.0,...,0.62967,59.370567,-15.1,-15.1,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0
1,p01,Morning,14.200000,0.0083,-1.0,3.931826,-1.0,14.200000,0.0083,-1.0,...,0.60048,56.618286,-14.4,-14.4,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0
2,p01,Morning,14.200000,0.0083,-1.0,3.931826,-1.0,14.200000,0.0083,-1.0,...,0.57963,54.652374,-13.9,-13.9,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0
3,p01,Morning,15.400000,0.0083,-1.0,3.931826,-1.0,15.400000,0.0583,-1.0,...,0.57546,54.259193,-13.8,-13.8,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0
4,p01,Morning,17.200001,0.0583,-1.0,3.931826,-1.0,17.200001,0.0583,-1.0,...,0.55878,52.686462,-13.4,-13.4,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0


In [7]:
train.drop(["p_num"], axis=1, inplace=True)
test.drop(["p_num"], axis=1, inplace=True)

/tmp/ipykernel_23/3900331671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(["p_num"], axis=1, inplace=True)
/tmp/ipykernel_23/3900331671.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(["p_num"], axis=1, inplace=True)


In [8]:
encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
train_array = encoder.fit_transform(train[["time"]])
test_array = encoder.transform(test[["time"]])

train_df = pd.DataFrame(train_array, columns=encoder.get_feature_names_out(['time']), index=train.index )
test_df = pd.DataFrame(test_array, columns=encoder.get_feature_names_out(['time']), index=test.index)

train = pd.concat([train.drop(columns=["time"]), train_df], axis=1)
test = pd.concat([test.drop(columns=["time"]), test_df], axis=1)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
train.head()

,bg0,insulin0,hr0,steps0,cals0,bg1,insulin1,hr1,steps1,cals1,...,bg35_cals35,insulin35_steps35,insulin35_hr35,insulin35_cals35,steps35_hr35,steps35_cals35,hr35_cals35,time_Afternoon,time_Morning,time_Night
0,13.900000,0.0083,-1.0,3.931826,-1.0,13.900000,0.0083,-1.0,3.931826,-1.0,...,-15.1,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0,0.0,1.0,0.0
1,14.200000,0.0083,-1.0,3.931826,-1.0,14.200000,0.0083,-1.0,3.931826,-1.0,...,-14.4,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0,0.0,1.0,0.0
2,14.200000,0.0083,-1.0,3.931826,-1.0,14.200000,0.0083,-1.0,3.931826,-1.0,...,-13.9,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0,0.0,1.0,0.0
3,15.400000,0.0083,-1.0,3.931826,-1.0,15.400000,0.0583,-1.0,3.931826,-1.0,...,-13.8,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0,0.0,1.0,0.0
4,17.200001,0.0583,-1.0,3.931826,-1.0,17.200001,0.0583,-1.0,3.931826,-1.0,...,-13.4,0.163957,-0.0417,-0.0417,-3.931826,-3.931826,1.0,0.0,1.0,0.0


In [10]:
def cross_validate(params, model_type, X, y, test_data, n_splits=2, n_bags = 1, random_state=42):
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(test_data))
    scores = []
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for fold, (train_index, valid_index) in enumerate(kfold.split(X,y)):
        for i in range(0,n_bags):
            X_train = X.iloc[train_index]
            y_train = y.iloc[train_index]
            X_val = X.iloc[valid_index]
            y_val = y.iloc[valid_index]
            
            X_train, y_train = dynamic_downsampling(X_train, y_train)
            
            if model_type == 'lgb':
                model = LGBMRegressor(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)],callbacks = callbacks)
            elif model_type == 'xgb':
                model = XGBRegressor(**params, early_stopping_rounds=100, n_jobs=4, random_state=42, max_bin=100000, device="cuda")
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose = 300)
            
            y_pred = model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            print(f"# Fold {fold}, bag {i}: RMSE={rmse:.5f}")
            scores.append(rmse)
            
            oof_preds[valid_index] += y_pred / n_bags
            test_preds += model.predict(test_data) / kfold.get_n_splits() / n_bags
            
            del model
            gc.collect()
            
        print(f"#RMSE mean: {np.mean(scores):.7f} (+- {np.std(scores):.7f})")

    return oof_preds, test_preds

In [11]:
def objective_lgb(trial):
    global lgb_test
    global lgb_oof
    params_lgbm = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 140),
        'learning_rate': trial.suggest_float('learning_rate', 0.15, 0.25),
        'n_estimators': trial.suggest_int('n_estimators', 3000, 4000),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 30),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1),
        'metric': 'root_mean_squared_error',
        'random_state': 42,
        'bagging_freq': 3,
        'verbose' : -1,
        'n_jobs': 4,
    }
    
    oof_preds_lgb, test_preds_lgb = cross_validate(params_lgbm, "lgb", X, y, test, n_splits = 5, n_bags = 3)

    rmse_score = np.sqrt(mean_squared_error(y, oof_preds_lgb))
    print(f"Trial {trial.number}, final RMSE: {rmse_score}")
    
    if trial.number == 0 or rmse_score < trial.study.best_value:
        lgb_test = test_preds_lgb
        lgb_oof = oof_preds_lgb
        print(f"Saved best test with RMSE score: {rmse_score}")
    
    return rmse_score


In [12]:
def dynamic_downsampling(X, y, seed=42):
    # Reset indices of X and y to ensure alignment
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)

    # Identify counts per class in p_num_int
    class_counts = X["p_num_int"].value_counts()
    min_class_count = class_counts.min()  # Minimum count across all classes

    # Downsample each class to the size of the smallest class
    downsampled_X = []
    downsampled_y = []

    for class_label, count in class_counts.items():
        class_data_X = X[X["p_num_int"] == class_label]
        class_data_y = y[class_data_X.index]

        # Downsample to the minimum class count if necessary
        if count > min_class_count:
            print("downsampling")
            class_sample_X, _ = train_test_split(
                class_data_X, train_size=min_class_count, random_state=seed
            )
            class_sample_y = class_data_y.loc[class_sample_X.index]
        else:
            print("No downsampling")
            class_sample_X = class_data_X
            class_sample_y = class_data_y

        downsampled_X.append(class_sample_X)
        downsampled_y.append(class_sample_y)

    # Concatenate all downsampled data
    X_balanced = pd.concat(downsampled_X).reset_index(drop=True)
    y_balanced = pd.concat(downsampled_y).reset_index(drop=True)

    return X_balanced, y_balanced

In [13]:
X = train
y = X.pop('bg+1:00')

callbacks = [log_evaluation(period=300), early_stopping(stopping_rounds=250)]

In [14]:
# Set up and run the Optuna study
#optuna.logging.set_verbosity(optuna.logging.WARNING)
#study = optuna.create_study(direction="minimize")
#study.optimize(objective_lgb, n_trials=8)
#print("Best parameters:", study.best_params)

In [15]:
best_params = {'num_leaves': 40, 'learning_rate': 0.0501091196307633, 'n_estimators': 4000, 'max_depth': 12, 'min_data_in_leaf': 15,
                'metric': 'root_mean_squared_error', 'random_state': 42,
               'device': 'gpu',
'verbose' : -1,
'n_jobs': 4,}
lgb_oof, lgb_test = cross_validate(best_params, "lgb", X, y, test, n_splits = 5, n_bags = 1)


downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
No downsampling


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 250 rounds
[300]	valid_0's rmse: 2.09098
Early stopping, best iteration is:
[90]	valid_0's rmse: 2.07492
# Fold 0, bag 0: RMSE=2.07492
#RMSE mean: 2.0749181 (+- 0.0000000)
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
No downsampling
Training until validation scores don't improve for 250 rounds
[300]	valid_0's rmse: 2.06422
Early stopping, best iteration is:
[74]	valid_0's rmse: 2.04946
# Fold 1, bag 0: RMSE=2.04946
#RMSE mean: 2.0621867 (+- 0.0127315)
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
downsampling
No downsampling
Training until validation scores don't improve for 250 rounds
[300]	valid_0's rmse: 2.06783
Early stopping, best iteration is:
[84]	

In [16]:
sample_sub = pd.read_csv("/kaggle/input/brist1d/sample_submission.csv")
sample_sub['bg+1:00'] = lgb_test.astype(np.float16)
sample_sub['id'] = test_id
sample_sub.drop(1)
sample_sub.to_csv('submission_lgb.csv', index=False)
print(sample_sub.head())

         id    bg+1:00
0  p01_8459   8.859375
1  p01_8460   6.832031
2  p01_8461   8.453125
3  p01_8462  11.578125
4  p01_8463   5.945312
